In [ ]:
!pip install ultralytics

from ultralytics import YOLO
import os
import shutil
import glob

In [24]:
DATA_PATH = os.path.join("..", "..", "data", "tracking")

TRAIN_DATA_PATH = os.path.join(DATA_PATH, "train")
TEST_DATA_PATH = os.path.join(DATA_PATH, "test")

TRAIN_IMAGES_PATH = os.path.join(TRAIN_DATA_PATH, "images")
TEST_IMAGES_PATH = os.path.join(TEST_DATA_PATH, "images")

TRAIN_IMAGES_PATH = os.path.abspath(TRAIN_IMAGES_PATH)
TEST_IMAGES_PATH = os.path.abspath(TEST_IMAGES_PATH)

TRAIN_LABELS_PATH = os.path.join(TRAIN_DATA_PATH, "labels")
TEST_LABELS_PATH = os.path.join(TEST_DATA_PATH, "labels")

os.makedirs(TRAIN_IMAGES_PATH, exist_ok=True)
os.makedirs(TEST_IMAGES_PATH, exist_ok=True)
os.makedirs(TRAIN_LABELS_PATH, exist_ok=True)
os.makedirs(TEST_LABELS_PATH, exist_ok=True)

# copy test images into test/images folder
MOT20_01_IMG_PATH = os.path.join(TEST_DATA_PATH, "01", "img1")
print(MOT20_01_IMG_PATH)
if os.path.exists(MOT20_01_IMG_PATH):
    if os.path.exists(TEST_IMAGES_PATH):
        print("Copying images")
        for img_file in glob.glob(os.path.join(MOT20_01_IMG_PATH, "*")):
            shutil.copy(img_file, TEST_IMAGES_PATH)
            shutil.copy(img_file, TRAIN_IMAGES_PATH)    # just a placeholder

MODELS_PATH = os.path.join("../models")

../../data/tracking/test/01/img1
Copying images


In [ ]:
det_txt_path = "det.txt"
image_width, image_height = 1920, 1080

with open(det_txt_path, "r") as f:
    lines = f.readlines()

for line in lines:
    values = line.strip().split(",")
    
    frame_id = int(values[0])
    x, y, w, h = map(float, values[2:6])
    confidence = float(values[6])
    class_id = 0

    x_center = (x + w / 2) / image_width
    y_center = (y + h / 2) / image_height
    w /= image_width
    h /= image_height

    label_filename = f"{frame_id:06d}.txt"  # like 000001.txt
    label_filepath = os.path.join(TEST_LABELS_PATH, label_filename)

    with open(label_filepath, "a") as label_file:
        label_file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

print("det.txt successfully converted to YOLO labels!")

det.txt successfully converted to YOLO labels!


In [26]:
# test the model

# create placeholders for train and val

test_yaml_path = os.path.join(TEST_DATA_PATH, "data.yaml")

test_yaml_content = f"""\
train: {TRAIN_IMAGES_PATH}
val: {TRAIN_IMAGES_PATH}
test: {TEST_IMAGES_PATH}
nc: 1
names: ["person"]
"""

# Write to the file
with open(test_yaml_path, "w") as f:
    f.write(test_yaml_content)

print(f"data.yaml created at {test_yaml_path}")

model_path = os.path.join(MODELS_PATH, "yolov9c_trained.pt")

model = YOLO(model_path)

results = model.val(data=test_yaml_path, split="test")

metrics = results.results_dict

print("\n--- Model Evaluation Metrics ---")
print(f"🔹 Precision: {metrics['metrics/precision(B)']:.4f}")
print(f"🔹 Recall: {metrics['metrics/recall(B)']:.4f}")
print(f"🔹 mAP@50: {metrics['metrics/mAP50(B)']:.4f}")
print(f"🔹 mAP@50-95: {metrics['metrics/mAP50-95(B)']:.4f}")
print(f"🔹 Fitness Score: {metrics['fitness']:.4f}")

data.yaml created at ../../data/tracking/test/data.yaml
Ultralytics 8.3.78 🚀 Python-3.8.18 torch-2.1.0 CUDA:0 (Quadro M1000M, 4035MiB)
YOLOv9c summary (fused): 156 layers, 25,320,019 parameters, 0 gradients, 102.3 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 2.78MB/s]
val: Scanning /home/floubsy2423/Projects/data/tracking/test/labels... 429 images, 0 backgrounds, 0 corrupt: 100%|██████████| 429/429 [00:00<00:00, 730.05it/s]

val: New cache created: /home/floubsy2423/Projects/data/tracking/test/labels.cache



/home/floubsy2423/Programs/anaconda3/envs/boostTrack/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 27/27 [00:58<00:00,  2.17s/it]


                   all        429      12610      0.718      0.807      0.813      0.599
Speed: 0.9ms preprocess, 116.2ms inference, 0.0ms loss, 3.7ms postprocess per image
Results saved to runs/detect/val4

--- Model Evaluation Metrics ---
🔹 Precision: 0.7183
🔹 Recall: 0.8068
🔹 mAP@50: 0.8132
🔹 mAP@50-95: 0.5994
🔹 Fitness Score: 0.6208
